# <font color='purple'>Drive</font>

In [215]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/My\ Drive/DataScienTest/rains

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/DataScienTest/rains


# <font color='Yellow'>Lib</font>

In [216]:


import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix

from os import listdir
from os.path import isfile, join

from joblib import dump, load

from google.colab import data_table


*   Mount de Google drive
*   On se place dans le repertoire dédié au projet



In [217]:
g_ = {}
g_['modeles'] = {}
g_['modeles']['dir'] = 'modeles'
g_['modeles']['load'] = {}

g_['current_vars'] = {}
g_['current_vars']['Date'] = None
g_['current_vars']['Location'] = None
g_['current_vars']['WindDir9am'] = None
g_['current_vars']['MinTemp'] = None
g_['current_vars']['MaxTemp'] = None
g_['current_vars']['Rainfall'] = None
g_['current_vars']['Evaporation'] = None
g_['current_vars']['Sunshine'] = None
g_['current_vars']['WindGustDir'] = None
g_['current_vars']['WindGustSpeed'] = None
g_['current_vars']['WindDir3pm'] = None
g_['current_vars']['WindSpeed9am'] = None
g_['current_vars']['WindSpeed3pm'] = None
g_['current_vars']['Humidity9am'] = None
g_['current_vars']['Humidity3pm'] = None
g_['current_vars']['Pressure9am'] = None
g_['current_vars']['Pressure3pm'] = None
g_['current_vars']['Cloud9am'] = None
g_['current_vars']['Cloud3pm'] = None
g_['current_vars']['Temp9am'] = None
g_['current_vars']['Temp3pm'] = None
g_['current_vars']['RainToday'] = None
g_['current_vars']['RainTomorrow'] = None

g_['y_var_name'] = 'RainTomorrow'

g_['windcat'] = ['WindGustDir','WindDir3pm','WindDir9am']
g_['windir'] = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
g_['windirnum'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

g_['drop_var'] = ['Date','Location','RainToday','RainTomorrow']

g_['X'] = pd.core.frame.DataFrame()
g_['y'] = pd.core.series.Series()

g_['vp'] = g_['fp'] = g_['vn'] = g_['fn'] = 0



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [218]:
def display_df(df:pd.DataFrame = None):

  global g_
  if type(df).__name__ == 'NoneType':
    df = g_['X'] 
  dfn = df.select_dtypes(include='number')
  display (dfn.head(1))
  dfo = df.select_dtypes(include='O')
  display (dfo.head(1))

In [219]:
def load_modeles():

  global g_
  fichiers = [f for f in listdir(g_['modeles']['dir']) if isfile(join(g_['modeles']['dir'], f))]
  for f in fichiers:
    name = f[:-4]
    g_['modeles']['load'][name] = load(g_['modeles']['dir']+'/'+f)



In [220]:
def cat_2_multi_num_horizon():

  global g_
  vars = g_['windcat']
  #vars = g_['X'].select_dtypes("O").columns.tolist()
  df = pd.DataFrame()

  i = 0
  for el in vars:
    if el == g_['y_var_name']:
      vars.pop(i)
      break
    else:
      i = i+1

  for var in vars:
    df = pd.concat ( [ pd.get_dummies(g_['X'][[var]]) , df ], axis = 1 )
    g_['X'] = g_['X'].drop([var],axis=1)

  g_['X'] = pd.concat([ g_['X'] , df ],axis=1)


In [221]:
def add_missing_cat2num_vars():
  
  global g_
  for wc in g_['windcat']:
    for wd in g_['windir']:
      var_ = wc+'_'+wd
      if not var_ in g_['X']:
        g_['X'][var_] = [0]

In [222]:
def replace_yn(df = None):

  global g_
  if type(df).__name__ == 'NoneType':
    df = g_['X']
  vars = df.select_dtypes("O").columns.tolist()
  for v in vars:
    df[v]= df[v].replace(['Yes','No'],[1, 0])

In [223]:
def drop_vars():

  global g_
  for var in g_['drop_var']:
    g_['X'] = g_['X'].drop([var],axis=1)

In [224]:
def prepare_df():

  global g_
  cat_2_multi_num_horizon()
  add_missing_cat2num_vars()
  drop_vars()
  replace_yn()
  #display_df()

In [225]:
def apply_modeles():

  global g_
  print (g_['modeles']['load'][g_['current_vars']['Location']])
  g_['y'] = g_['modeles']['load'][g_['current_vars']['Location']].predict(g_['X'])

In [226]:
def apply():

  global g_
  load_modeles()
  prepare_df()
  apply_modeles()

In [227]:

def drop_nan(colrow:str,how:str,df:pd.DataFrame = None):
  
  global g_
  if colrow == 'row':
    cr = 0
  elif colrow == 'col':
    cr = 1
  else:
    cr = None
  if type(df).__name__ == 'NoneType':
    df = g_['X']
  df.dropna(axis = cr, how = how, inplace = True)


In [228]:
def clean_df():
  
  global g_
  df = g_['X']

  av_col = list(df.columns.values)
  av_shape = df.shape

  drop_nan( 'col','all', df  )
  drop_nan( 'row','any', df  )

  ap_col = list(df.columns.values)
  ap_shape = df.shape

  for x in ap_col:
    av_col.remove(x)

  col_supr_nb = av_shape[1] - ap_shape[1]
  row_supr_nb = av_shape[0] - ap_shape[0]


# <font color='green'>Test one row</font>

In [229]:
def set_g_vars():

  global g_
  g_['current_vars']['Date'] = '2012-01-13'
  g_['current_vars']['Location'] = 'Perth'
  g_['current_vars']['WindDir9am'] = 'ENE'
  g_['current_vars']['MinTemp'] = 23.8
  g_['current_vars']['MaxTemp'] = 34.9
  g_['current_vars']['Rainfall'] = 0
  g_['current_vars']['Evaporation'] = 13.2
  g_['current_vars']['Sunshine'] = 12.4
  g_['current_vars']['WindGustDir'] = 'NE'
  g_['current_vars']['WindGustSpeed'] = 39
  g_['current_vars']['WindDir3pm'] = 'ENE'
  g_['current_vars']['WindSpeed9am'] = 19
  g_['current_vars']['WindSpeed3pm'] = 15
  g_['current_vars']['Humidity9am'] = 51
  g_['current_vars']['Humidity3pm'] = 37
  g_['current_vars']['Pressure9am'] = 1013
  g_['current_vars']['Pressure3pm'] = 1009.4
  g_['current_vars']['Cloud9am'] = 0
  g_['current_vars']['Cloud3pm'] = 4
  g_['current_vars']['Temp9am'] = 27.6
  g_['current_vars']['Temp3pm'] = 33.1
  g_['current_vars']['RainToday'] = 'No'
  g_['current_vars']['RainTomorrow'] = 'No'

  g_['X'] = pd.DataFrame(g_['current_vars'], index=[0])

In [230]:
set_g_vars()
apply()

print ('y: ',g_['y'])
print ('RainTomorrow: ',g_['current_vars']['RainTomorrow'])

LogisticRegression(max_iter=400, solver='newton-cg')
y:  [0]
RainTomorrow:  No


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


# <font color='blue'>Test on complete file</font>

In [231]:
def load_csv():

  global g_
  g_['data_link'] = 'input/rains.csv'
  g_['X'] = pd.read_csv(g_['data_link'])

In [232]:
def list_location():

  global g_
  g_['location_list'] = set(g_['X']['Location'])


In [233]:
def traite_result():

  global g_
  (g_['vn'], g_['fp']), (g_['fn'], g_['vp'])  = confusion_matrix(g_['RainTomorrow'],g_['y'])



In [234]:
def init():

  global g_
  result = {}
  for loc in g_['location_list']:
    result[loc] = {}

In [235]:
def traite_df():

  global g_
  #g_['Location'] = 'Canberra'
  g_['X'] = g_['X'].loc[g_['X']['Location'] == g_['Location']]
  g_['X'] = g_['X'].loc[g_['X']['RainToday'] == 'No']
  g_['current_vars']['Location'] = g_['Location']



In [236]:
def var_supervised():

  global g_
  g_['RainTomorrow'] = pd.DataFrame(g_['X']['RainTomorrow'])
  replace_yn(g_['RainTomorrow'])

In [237]:
for g_['Location'] in ['Adelaide', 'Brisbane','Canberra','GoldCoast','Melbourne', 'Perth','Sydney']:#,'Newcastle']:

  print (g_['Location'])

  load_csv()
  traite_df()
  clean_df()
  var_supervised()


  apply()
  traite_result()


  print (
        'vp=', g_['vp'],
        'fp=', g_['fp'],
        'vn=', g_['vn'],
        'fn=', g_['fn'],
        "\n")


Adelaide
LogisticRegression(max_iter=400, solver='newton-cg')
vp= 76 fp= 24 vn= 689 fn= 67 

Brisbane
LogisticRegression(max_iter=400, solver='newton-cg')
vp= 146 fp= 77 vn= 1873 fn= 215 

Canberra


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


LogisticRegression(max_iter=400, solver='newton-cg')
vp= 64 fp= 22 vn= 700 fn= 73 

GoldCoast
LogisticRegression(max_iter=400, solver='newton-cg')
vp= 138 fp= 80 vn= 1676 fn= 203 

Melbourne


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


LogisticRegression(max_iter=400, solver='newton-cg')
vp= 113 fp= 49 vn= 1062 fn= 161 

Perth
LogisticRegression(max_iter=400, solver='newton-cg')
vp= 179 fp= 43 vn= 2079 fn= 132 

Sydney


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


LogisticRegression(max_iter=400, solver='newton-cg')
vp= 91 fp= 63 vn= 1004 fn= 120 



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
